CNN version 1.0
=============
Using the preprocessed Data
------------

Building upon the previous cnn version. 

This notebook uses a more sophisticated and a deeper cnn than the previous

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

# Task 1: Load the data from the pickle file

In [9]:
'''Set the constant paths here'''
# set the paths required for the script to work:
root = "../Data/"
pickle_file_path = root + "Data_train_final.pickle"
log_path = root + ".logs"

Retrieve the saved data from the pickle file

In [3]:
Formatted_Data = {}
with open(pickle_file_path, "rb") as f:
    Formatted_Data = pickle.load(f)

In [4]:
len(Formatted_Data)

7

In [5]:
# recollect the fragmented data

labels_mappings = Formatted_Data['label_mapping']
print(labels_mappings)

train_dataSet = Formatted_Data['train_data']
train_labels  = Formatted_Data['train_labels']

cv_dataSet = Formatted_Data['cv_data']
cv_labels = Formatted_Data['cv_labels']

test_dataSet = Formatted_Data['test_data']
test_labels  = Formatted_Data['test_labels']

print(train_dataSet.shape, cv_dataSet.shape, test_dataSet.shape)
print(train_labels.shape, cv_labels.shape, test_labels.shape)

{'dog': 1, 'cat': 0}
(17500, 32, 32, 3) (2500, 32, 32, 3) (5000, 32, 32, 3)
(17500, 2) (2500, 2) (5000, 2)


# Task 2: Spawn the CNN 

In [6]:
# The accuracy function to calculate the accuracy of the classifier

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [7]:
# define the constants required hence forth:
batch_size = 100
patch_size = 3
num_hidden = 256

image_size = 32
num_labels = len(labels_mappings)
num_channels = 3 # rgb images

### The CNN tensorflow graph 

In [93]:
def weightVariable(shape, Wname="W"):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial, name=Wname)
        
def biasVariable(shape, Bname="B"):    
    initial = tf.zeros(shape)
    return tf.Variable(initial, name=Bname)
        
def addConvLayer(inp, shape, act_fun=tf.nn.relu):
    with tf.name_scope("Conv_Layer"):
        with tf.name_scope("Parameters"):
            # create the weight variable:
            layer_weight = weightVariable(shape, Wname="Filters")

            # create bias variable:
            layer_bias = biasVariable([shape[-1]], Bname="Filter_biases")

        return act_fun(tf.nn.conv2d(inp, layer_weight, [1, 1, 1, 1], padding="SAME") + layer_bias)
    
def addPoolLayer(inp, kernel, stride):
    return tf.nn.max_pool(inp, ksize=kernel, strides=stride, padding='SAME')
    
def addFCLayer(inp, shape, act_fun=tf.nn.relu):
    with tf.name_scope("Fully_Connected_Layer"):
        layer_weight = weightVariable(shape)
        layer_bias = biasVariable([shape[-1]])

        if(act_fun == None): 
            return tf.matmul(inp, layer_weight) + layer_bias
        # else:
        return act_fun(tf.matmul(inp, layer_weight) + layer_bias)

In [103]:
graph = tf.Graph()

''' Neural network name: BotNet
    Size: medium
    conf: conv => conv => pool => conv => pool => fc => fc => softmax'''

with graph.as_default():
    
    # Input data.
    with tf.name_scope("INPUTS"):
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

        tf_valid_dataset = tf.constant(cv_dataSet, name="cross_validation_dataset")
        tf_test_dataset = tf.constant(test_dataSet, name="test_dataset")
  
    # Model.
    def model(data):
        # CONV LAYER 
        # layer has 8 (3 x 3) convoluting filters
        layer_1 = addConvLayer(data, [patch_size, patch_size, num_channels, 8], tf.nn.sigmoid)
    
        # CONV LAYER
        # layer has 16 (3 x 3) convoluting filters
        layer_2 = addConvLayer(layer_1, [patch_size, patch_size, 8, 16], tf.nn.sigmoid)
        
        # POOL LAYER
        # will not require any data variable or parameter holder
        # a simple 2 x 2 max_pooling operation
        layer_3 = addPoolLayer(layer_2, [1, 2, 2, 1], [1, 2, 2, 1])
        
        # CONV LAYER
        # layer has 32 (3 x 3) convoluting filters
        layer_4 = addConvLayer(layer_3, [patch_size, patch_size, 16, 32], tf.nn.sigmoid)
        
        # POOL LAYER
        # will not require any data variable or parameter holder
        # a simple 2 x 2 max_pooling operation
        layer_5 = addPoolLayer(layer_4, [1, 2, 2, 1], [1, 2, 2, 1])
        
        # FC LAYER
        # input dimension will be: 8 * 8 * 32
        # length, width calculation: 32 =first pool=> 16 =second pool=> 8
        # depth calculation: 32 ...(same as the last conv output)
        # output dimension: 128 ...(equal to number of hidden cells)
        # number of hidden cells: 128
        # reshape the tensor before passing it to the FC layers
        shape = layer_5.get_shape().as_list()
        reshape = tf.reshape(layer_5, [shape[0], shape[1] * shape[2] * shape[3]])
        layer_6 = addFCLayer(reshape, [8 * 8 * 32, num_hidden], tf.nn.sigmoid)
        
        # FC LAYER
        # input dimension will be: 256
        # output dimension will be: 256
        # number of hidden cells: 256
        layer_7 = addFCLayer(layer_6, [num_hidden, num_hidden], tf.nn.sigmoid)
        
        # Fianl output layer for the neural network
        layer_8 = addFCLayer(layer_7, [num_hidden, num_labels], tf.nn.sigmoid)
        
        return layer_8
  
    # Training computation.
    logits = model(tf_train_dataset)
    
    with tf.name_scope("Loss"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
        tf.scalar_summary("loss", loss)
        
    # Optimizer.
    with tf.name_scope("train_step"):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    with tf.name_scope("train_prediction"):
        train_prediction = tf.nn.softmax(logits)
    # validation_prediction = tf.nn.softmax(model(tf_valid_dataset))

### Graph Excuter Session module

In [ ]:
num_steps = 5001
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
with tf.Session(graph=graph) as session:
    train_writer = tf.train.SummaryWriter(os.path.join(log_path, "train"), session.graph)
    merged = tf.merge_all_summaries()
    tf.global_variables_initializer().run()
    print('Initialized')
  
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataSet[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step % 500 == 0):
            summary = session.run(merged, feed_dict=feed_dict)
            train_writer.add_summary(summary, step)
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            
    #print('Cross Validation_Set Accuracy: %1f%%' % accuracy(validation_prediction.eval(), cv_labels))

Initialized
Minibatch loss at step 0: 0.727073
Minibatch accuracy: 40.0%
